Imports

In [1]:
from __future__ import division
import matplotlib.pyplot as plt

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import math
import numpy as np
import json 
import sys

sys.path.insert(0, './car')
from nearest_neighbour import nearest_neighbour
sys.path.insert(0, './car/detector')
import aux
import server

import argparse
import torch
import cv2
import pyzed.sl as sl

from threading import Lock, Thread
import time
from datetime import datetime

sys.path.insert(0, './yolov5')
from models.experimental import attempt_load
from utils.general import check_img_size, non_max_suppression
from utils.torch_utils import select_device

Welcome!


Globale Variablen

In [2]:
lock = Lock()
run_signal = False
exit_signal = False

pylons = ["blue","green","orange","pink","yellow"]

bizzi Waiting

In [3]:
bizzi = True

Global Cone Cache

In [4]:
# colors in BGR
Cache = {
    "blue": {
        "classId": 0,
        "color": (255,0,0),
        "items": {}
    },
    "green": {
        "classId": 1,
        "color": (0,255,0),
        "items": {}
    },
    "red": {
        "classId": 2,
        "color": (0,0,255),
        "items": {}
    },
    "pink": {
        "classId": 3,
        "color": (255,0,255),
        "items": {}
    },
    "yellow": {
        "classId": 4,
        "color": (0,255,255),
        "items": {}
    },
    "self": {
        "classId": [],
        "color": (0,0,0),
        "items": {
            "0":{
                "translation":{
                    "x": 0,
                    "y": 0,
                    "z": 0,
                },
                "orientation": {
                    "x": 0,
                    "y": 0,
                    "z": 0,
                    "w": 0,
                },
                "timestamp": 0
            }
        }
    }
}
server.setCache(Cache)

In [10]:
def init_cam():
    zed = sl.Camera()
    input_type = sl.InputType()
    if opt["svo"] is not None:
        input_type.set_from_svo_file(opt["svo"])

    # Create a InitParameters object and set configuration parameters
    init_params = sl.InitParameters(input_t=input_type, svo_real_time_mode=True)
    init_params.camera_resolution = sl.RESOLUTION.HD1080
    init_params.coordinate_units = sl.UNIT.CENTIMETER
    init_params.coordinate_system = sl.COORDINATE_SYSTEM.LEFT_HANDED_Y_UP
    # init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # QUALITY
    init_params.depth_mode = sl.DEPTH_MODE.QUALITY #PERFORMANCE  # QUALITY
    # init_params.depth_maximum_distance = 20
    status = zed.open(init_params)
    return zed, status, init_params

In [11]:
def debug_camera_settings(zed):
    print("BRIGHTNESS: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.BRIGHTNESS))) # 0 - 8
    print("CONTRAST: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.CONTRAST))) # 0 - 8
    print("HUE: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.HUE))) # 0 - 11
    print("SATURATION: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.SATURATION))) # 0 - 8
    print("SHARPNESS: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.SHARPNESS))) # 0 - 8
    print("GAMMA: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.GAMMA))) # 1 - 9
    print("GAIN: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.GAIN))) # 0 - 100
    print("EXPOSURE: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.EXPOSURE))) # 0 - 100
    print("AEC_AGC: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.AEC_AGC))) # True | False
    print("AEC_AGC_ROI: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.AEC_AGC_ROI))) # True | False
    print("WHITEBALANCE_TEMPERATURE: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.WHITEBALANCE_TEMPERATURE))) # 2800 - 6500 (100+)
    print("WHITEBALANCE_AUTO: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.WHITEBALANCE_AUTO))) # True | False
    print("LED_STATUS: {0}".format(zed.get_camera_settings(sl.VIDEO_SETTINGS.LED_STATUS))) # 0 - 1

In [12]:
def enable_tracking(zed):
    
    # If the camera is static, uncomment the following line to have better performances and boxes sticked to the ground.
    positional_tracking_parameters = sl.PositionalTrackingParameters()
    # positional_tracking_parameters.set_as_static = True
    # positional_tracking_parameters.initial_world_transform = True

    initial_position = sl.Transform()
    initial_translation = sl.Translation()
    # initial_translation.init_vector(0,18,-75)
    initial_position.set_translation(initial_translation)
    positional_tracking_parameters.set_initial_world_transform(initial_position)

    zed.enable_positional_tracking(positional_tracking_parameters)


In [13]:
def enable_object_detection(zed):
    
    obj_param = sl.ObjectDetectionParameters()
    obj_param.detection_model = sl.DETECTION_MODEL.CUSTOM_BOX_OBJECTS
    # obj_param.enable_tracking = True
    # obj_param.enable_mask_output = True
    zed.enable_object_detection(obj_param)


In [14]:
def update_position(zed):
    # Get the pose of the camera relative to the world frame
    state = zed.get_position(cam_w_pose, sl.REFERENCE_FRAME.WORLD)
    # Display translation and timestamp
    py_translation = sl.Translation()
    tx = round(cam_w_pose.get_translation(py_translation).get()[0], 3)
    ty = round(cam_w_pose.get_translation(py_translation).get()[1], 3)
    tz = round(cam_w_pose.get_translation(py_translation).get()[2], 3)
    # print("Translation: tx: {0}, ty:  {1}, tz:  {2}, timestamp: {3}".format(tx, ty, tz, cam_w_pose.timestamp.get_seconds()))
    #Display orientation quaternion
    py_orientation = sl.Orientation()
    ox = round(cam_w_pose.get_orientation(py_orientation).get()[0], 3)
    oy = round(cam_w_pose.get_orientation(py_orientation).get()[1], 3)
    oz = round(cam_w_pose.get_orientation(py_orientation).get()[2], 3)
    ow = round(cam_w_pose.get_orientation(py_orientation).get()[3], 3)

    ex = round(cam_w_pose.get_euler_angles()[0],3)
    ey = round(cam_w_pose.get_euler_angles()[1],3)
    ez = round(cam_w_pose.get_euler_angles()[2],3)

    vx = round(cam_w_pose.get_rotation_vector()[0],3)
    vy = round(cam_w_pose.get_rotation_vector()[1],3)
    vz = round(cam_w_pose.get_rotation_vector()[2],3)

    # print("Translation: tx: {0}, ty:  {1}, tz:  {2}, timestamp: {3} | Orientation: ox: {4}, oy:  {5}, oz: {6}, ow: {7}".format(tx, ty, tz, cam_w_pose.timestamp.get_seconds(),ox, oy, oz, ow))
    # roll_x, pitch_y, yaw_z = euler_from_quaternion(ox,oy,oz,ow)
    # (x, y, z, w):
    Cache["self"]["items"]["0"] = {
        "translation":{
            "x": tx,
            "y": ty,
            "z": tz,
        },
        "orientation": {
            "x": ox,
            "y": oy,
            "z": oz,
            "w": ow,
        },
        "euler": {
            "x": ex,
            "y": ey,
            "z": ez,
        },
        "rvect": {
            "x": vx,
            "y": vy,
            "z": vz,
        },
        "timestamp": cam_w_pose.timestamp.get_seconds()
    }

In [ ]:
def init_network(img_size,weights,device):
    device = select_device()
    half = device.type != 'cpu'  # half precision only supported on CUDA
    imgsz = img_size

    # Load model
    model = attempt_load(weights, device,False)  # load FP32
    bizzi = False
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size
    if half:
        model.half()  # to FP16

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once



In [17]:
def process_image():
    img, ratio, pad = aux.img_preprocess(image_net, device, half, imgsz)
    pred = model(img)[0]
    det = non_max_suppression(pred, conf_thres, iou_thres)
    detections = aux.detections_to_custom_box(det, img, image_net)
    zed.ingest_custom_box_objects(detections)
    zed.retrieve_objects(objects, obj_runtime_param)
    # if objects.is_new :
    obj_array = objects.object_list

In [16]:
def get_image_and_process():
    update_position(zed)
    zed.retrieve_image(image_left_tmp, sl.VIEW.LEFT)
    image_net = image_left_tmp.get_data()

    process_image()

    tempTime = str(datetime.utcnow().strftime('%H:%M:%S.%f')[:-3])[0:8]
    if timeName != tempTime:
        print("FPS: {0}".format(str(timeNum)))
        timeNum = 0
        timeName = tempTime
    timeNum = timeNum + 1

    blue_pylons = []
    red_pylons = []
    if len(obj_array) > 0:
        for obj in obj_array:
            if not math.isnan(obj.position[0]):
                if obj.tracking_state==sl.OBJECT_TRACKING_STATE.OK:

                    color = (0,0,0)

                    def position_to_object(pos):
                        return {"x":pos[0],"y":pos[1],"z":pos[2]}

                    def compare_position_and_update(old,new,limit=0.5):
                        a = np.array((old["x"],old["y"],old["z"]))
                        b = np.array((new["x"],new["y"],new["z"]))
                        dist = np.linalg.norm(a-b)
                        return old if dist > limit else new

                    key = ""

                    if(obj.raw_label == Cache["blue"]["classId"]):
                        key = "blue"
                    elif(obj.raw_label == Cache["green"]["classId"]):
                        key = "green"
                    elif(obj.raw_label == Cache["red"]["classId"]):
                        key = "red"
                    elif(obj.raw_label == Cache["pink"]["classId"]):
                        key = "pink"
                    elif(obj.raw_label == Cache["yellow"]["classId"]):
                        key = "yellow"
                    else:
                        print("Found pylon out of identity")
                        return


                    color = Cache[key]["color"]
                    if obj.id in Cache[key]["items"]:
                        Cache[key]["items"][obj.id] = compare_position_and_update(
                            Cache[key]["items"][obj.id],
                            position_to_object(obj.position)
                        )
                    else:
                        Cache[key]["items"][obj.id] = position_to_object(obj.position)

                    font = cv2.FONT_HERSHEY_SIMPLEX
                    org = (int(obj.bounding_box_2d[0][0]),int(obj.bounding_box_2d[0][1]))
                    fontScale = .5
                    thickness = 1
                    image_net = cv2.putText(image_net, "(id: {2} | class: {3}) {0} x {1}".format(
                        round(obj.position[0]-30),
                        round(obj.position[2]),
                        obj.id,
                        key
                    ), org, font, fontScale, color, thickness, cv2.LINE_AA)
                    # print("draw")
                    cv2.line(image_net, (int(obj.bounding_box_2d[0][0]),int(obj.bounding_box_2d[0][1])),(int(obj.bounding_box_2d[1][0]),int(obj.bounding_box_2d[1][1])), color, 1) 
                    cv2.line(image_net, (int(obj.bounding_box_2d[1][0]),int(obj.bounding_box_2d[1][1])),(int(obj.bounding_box_2d[2][0]),int(obj.bounding_box_2d[2][1])), color, 1) 
                    cv2.line(image_net, (int(obj.bounding_box_2d[2][0]),int(obj.bounding_box_2d[2][1])),(int(obj.bounding_box_2d[3][0]),int(obj.bounding_box_2d[3][1])), color, 1) 
                    cv2.line(image_net, (int(obj.bounding_box_2d[3][0]),int(obj.bounding_box_2d[3][1])),(int(obj.bounding_box_2d[0][0]),int(obj.bounding_box_2d[0][1])), color, 1) 

        # print("show")

        temp_pylons = []

        def copy_item(item,i,cls):
            return {"x":item["x"],"y":item["y"],"color":cls,"id":str(i)}

        for item in Cache["blue"]["items"]:
            temp_pylons.append(copy_item(Cache["blue"]["items"][item],item,"blue"))

        for item in Cache["green"]["items"]:
            temp_pylons.append(copy_item(Cache["green"]["items"][item],item,"blue"))

        for item in Cache["red"]["items"]:
            temp_pylons.append(copy_item(Cache["red"]["items"][item],item,"red"))

        for item in Cache["pink"]["items"]:
            temp_pylons.append(copy_item(Cache["pink"]["items"][item],item,"red"))

        # print(temp_pylons)

        route = None
        neighbours = None
        curve = None
        pylons = None
        blue_curved = None
        red_curved = None

        if(len(temp_pylons)):
            route,neighbours,curve,pylons,blue_curved,red_curved = nearest_neighbour(temp_pylons,False)

        # json_object = json.dumps(Cache)
        json_object = json.dumps({
            "neighbours": neighbours,
            "route": route,
            "curve": curve,
            "pylons": pylons,
            "blueCurved": blue_curved,
            "redCurved": red_curved,
        })
        server.setImage(image_net[:,:,:3])
        now = datetime.now()
        if secCache != now.second:
            secCache  = now.second
            server.sendWebsocketMessage("left-eye:"+str(aux.image_to_base64(image_net[:,:,:3])))
        # server.sendWebsocketMessage("chart:"+str(aux.image_to_base64(image)))
        server.setPositions(json_object)
        # server.sendWebsocketMessage("pylons:"+json_object)
        # else:
        #     plotPylons([], [],image_net)

Plot Thread

In [21]:
def plt_thread(weights, img_size, conf_thres=0.2, iou_thres=0.45):
    global image_net, exit_signal, run_signal, detections
    print("Initializing Camera...")
    
   
    zed, status, init_params = init_cam()           # Init Cam
    runtime_params = sl.RuntimeParameters()         # Getting Runtime Parameters
    #debug_camera_settings(zed)                      # Printing all camera settings to console
    server.setZed(zed,sl)                           # Setting Zed inside the Server to change settings later via API

    # Making sure Camera status is ok
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit()

    # Preparing variable
    image_left_tmp = sl.Mat()
    timeName = ""
    timeNum = 0
    secCache = 0

    print("Initialized Camera")

    enable_tracking(zed)
    enable_object_detection(zed)
    
    objects = sl.Objects()
    obj_runtime_param = sl.ObjectDetectionRuntimeParameters()
    # obj_runtime_param.detection_confidence_threshold = 25

    # Display
    cam_w_pose = sl.Pose()

    print("Intializing Network...")
    init_network(img_size,weights,device)
    
    while not exit_signal:
        if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
            get_image_and_process()
        else:
            print("failed")
    print("done")

    zed.close()


Main Function

In [23]:
opt = {
    "weights": "best.pt",
    "svo": None,
    "img_size": 416,
    "conf_thres": 0.4,
}

plot_thread = Thread(target=plt_thread, kwargs={
    'weights': opt["weights"],
    'img_size': opt["img_size"],
    "conf_thres": opt["conf_thres"]
})
plot_thread.start()

Initializing Camera...


YOLOv5 🚀 2022-11-7 Python-3.8.10 torch-1.12.0a0+2c916ef.nv22.3 CUDA:0 (Xavier, 6855MiB)



CAMERA STREAM FAILED TO START
Initialized Camera
Intializing Network...


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_57104/2470244653.py", line 35, in plt_thread
  File "/tmp/ipykernel_57104/2839698965.py", line 7, in init_network
NameError: name 'weights' is not defined


In [9]:
exit_signal = True
plot_thread.join()

In [7]:
server.startServers()

Using wlan0's ip: 192.168.0.19
Web Server started @ http://192.168.0.19:8080
WebSocket Server started @ http://192.168.0.19:8081


Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs


FPS: 0
FPS: 2
FPS: 4
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 2
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 2
FPS: 1
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 1
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 2
FPS: 2
FPS: 2
FPS: 3
FPS: 3
FPS: 2
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 1
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 2
FPS: 3
FPS: 3
FPS: 3
FPS: 3


KeyboardInterrupt: 

In [ ]:
server.stopServer()